In [1]:
print("Hello....")

Hello....


In [2]:
%pwd

'c:\\Users\\tejas\\OneDrive\\Desktop\\Chatbot_GenAI\\genai_chatbot\\research'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\tejas\\OneDrive\\Desktop\\Chatbot_GenAI\\genai_chatbot'

In [6]:
#Extract the data from the PDF file
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
data_extraction=load_pdf_file(data='Data/')

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(data_extraction)
print("Length of Text Chunks is:", len(text_chunks))

Length of Text Chunks is: 148


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# Download the Embedding model from Hugging Face
def huggingface_embedding_model():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings=huggingface_embedding_model()

C:\Users\tejas\AppData\Local\Temp\ipykernel_17460\3671146463.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Anaconda\envs\chatbot_genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
test_embedding_model=embeddings.embed_query("this is tejaswi")
test_embedding_model

[-0.0381503589451313,
 0.11391490697860718,
 -0.08430175483226776,
 0.026194293051958084,
 -0.033716291189193726,
 -0.04760805144906044,
 0.057630300521850586,
 -0.039994191378355026,
 0.05761007219552994,
 0.02264462225139141,
 0.05278363451361656,
 -0.05643414705991745,
 0.032788313925266266,
 0.054703567177057266,
 -0.007925480604171753,
 -0.03318250924348831,
 -0.02648223005235195,
 -0.014706172049045563,
 0.06408940255641937,
 -0.08025839179754257,
 -0.00765236746519804,
 0.09792744368314743,
 0.016366181895136833,
 -0.069583959877491,
 0.029192784801125526,
 -0.02192474715411663,
 -0.005268258973956108,
 0.08286932110786438,
 0.024557508528232574,
 -0.03930988907814026,
 -0.032516490668058395,
 0.05777914822101593,
 -0.0773068517446518,
 -0.04087197780609131,
 -0.07095035910606384,
 0.009683963842689991,
 -0.02232145331799984,
 0.048074182122945786,
 -0.010110610164701939,
 0.0108334394171834,
 -0.09665190428495407,
 -0.035508446395397186,
 -0.027590328827500343,
 -0.003894374705

In [14]:
print("The length of the vector is: ", len(test_embedding_model))

The length of the vector is:  384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Pinecone_API=os.environ.get("PINECONE_API_KEY")
# OpenAI_API=os.environ.get("OPENAI_API_KEY")
# print(OpenAI_API)
# print(Pinecone_API)

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=Pinecone_API)

In [ ]:
# creating an index in the pinecone vector base
index_name = "chatbot"

pc.create_index(name=index_name, 
                dimension=384, 
                metric='cosine', 
                spec=ServerlessSpec(cloud='aws',
                                    region='us-east-1'
                                    )
)

{
    "name": "chatbot-genai",
    "metric": "cosine",
    "host": "chatbot-genai-m1o0ypu.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
# import os
# os.environ["PineconeAPIkey"]=Pinecone_API
# os.environ["OpenaiAPIkey"]=OpenAI_API

In [20]:
from langchain_pinecone import PineconeVectorStore

# Convert each chunk into a vector and store it in the index
vectors = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


In [35]:
retrieve_answers=vectors.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [36]:
answers_generated = retrieve_answers.invoke("explain about electronic health records?")
answers_generated

[Document(id='6ee94fdd-612b-464c-8084-898fbf4bef95', metadata={'creationdate': '2015-06-16T22:33:28-04:00', 'creator': 'TeX', 'moddate': '2015-06-16T22:33:38-04:00', 'page': 4.0, 'page_label': '5', 'producer': 'pdfTeX-1.40.15', 'source': 'Data\\HDA.pdf', 'total_pages': 18.0}, page_content='Electronic health records (EHRs) contain a digitized versi on of a patient’s medical history. It\nencompasses a full range of data relevant to a patient’s care such as demographics, problems, med-\nications, physician’s observations, vital signs, medical history, laboratory data, radiology reports,\nprogress notes, and billing data. Many EHRs go beyond a patie nt’s medical or treatment history and\nmay contain additional broader perspectives of a patient’s care. An important property of EHRs is'),
 Document(id='d7e06043-fef6-4a3d-a4ca-074176e5c955', metadata={'creationdate': '2015-06-16T22:33:28-04:00', 'creator': 'TeX', 'moddate': '2015-06-16T22:33:38-04:00', 'page': 4.0, 'page_label': '5', 'produce

In [23]:
# Load the existing index from pinecone vector store
load_index_vectos = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)
load_index_vectos

In [24]:
retrieval = load_index_vectos.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [25]:
answers = retrieval.invoke("explain about electronic health records?")

In [26]:
answers

[]

In [27]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.2, max_tokens = 200, model='gpt-3.5-turbo')

C:\Users\tejas\AppData\Local\Temp\ipykernel_17460\2399839734.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.2, max_tokens = 200, model='gpt-3.5-turbo')


In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
"You are a helpful AI assistant. You must answer questions only using the information provided in the PDF content. Do not use any external knowledge, assumptions, or personal opinions." 

"If the answer is not explicitly available in the PDF, say: "

"I'm sorry, the answer to that question is not available."

"Do not attempt to guess, infer, or hallucinate."

"Always quote or refer to the relevant part of the document when answering."

"\n\n"

"{context}"

)

In [29]:
prompt =  ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [30]:
question_and_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retrieval, question_and_answer_chain)

In [33]:
response = rag_chain.invoke(
    {
        "input": "explain about java?"
    }
)

In [34]:
response["answer"]

"I'm sorry, the answer to that question is not available."